In [ ]:
from sctoolbox.utilities import bgcolor

# 0B - Receptor-Ligand Analysis
<hr style="border:2px solid black"> </hr>

## 1 - Description

**Note: Requires notebook 04_clustering to be run prior to this notebook.**

Receptor-ligand interactions play a crucial role in mediating cellular communication and signaling processes in biological systems. In the context of single-cell data analysis, studying receptor-ligand interactions provides insights into cell-cell communication networks and their impact on various cellular functions and behaviors.  
This notebook provides the tools to investigate and visualize the ligand-receptor interactions between conditions and/or clusters.

--------------

## 2 - Setup

In [ ]:
import scanpy
import sctoolbox.receptor_ligand as rl
import sctoolbox.utilities as utils
from sctoolbox import settings
utils.settings_from_config("config.yaml", key="0B")

--------------

## 3 - Load adata
The dataset must be clustered for a receptor-ligand analysis.

In [ ]:
adata = utils.load_h5ad("anndata_4.h5ad")
display(adata)

--------------

## 4 - General input

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

# Path to receptor ligand database
db_path = f'http://tcm.zju.edu.cn/celltalkdb/download/processed_data/human_lr_pair.txt'

# Comput interactions
cluster_col = "clustering"
normalize = 1000 # Correct clusters to given size.
gene_col = None # Column that holds gene name /id. Set to None to use index
overwrite = False # Overwrite existing interaction table

# Plotting

dpi = 100

## Violin
violin_min_perc = 0 # Minimum percentage of cells in a cluster that express the respective gene.
violin_out_suffix = "violin" # Suffix of output file
violin_figsize = (5, 30) # Figure size

## Network
net_min_perc = 0 # Minimum percentage of cells in a cluster that express the respective gene.
net_interaction_score = 0 # Interaction score must be above this threshold for the interaction to be counted in the graph.
net_interaction_perc = 90 # Select interaction scores above or equal to the given percentile. Will overwrite parameter interaction_score.
net_out_suffix = "network" # Suffix of output file
net_title = None # Plot title
net_color_min = 0 # Min value for color range
net_color_max = None # Max value for color range
net_restrict_to = [] # Only show given clusters provided in list.
net_show_count = True # Show the interaction count

## Connection plot
con_restrict_to = None # Restrict plot to given cluster names.
con_figsize = (5, 10) # Figure size
con_out_suffix="connectionPlot" # Suffix of output file
con_title = None # Plot title
con_filter = con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 50 & ligand_percent >= 50" # Conditions to filter the interaction table
con_lw_multiplier = 2 # Linewidth multiplier
con_wspace = 0.4 # Width between plots
con_lig_whitelist = None # List of ligand genes that should be exclusively shown.
con_rec_whitelist = None # List of receptor genes that should be exclusively shown.

## Filter for interaction table
tab_min_perc = 0
tab_interaction_score = 0
tab_interaction_perc = 90
tab_out_suffix = "interaction_table"
group_a = None
group_b = None

__Use `help(<function_name>)` for information on the parameters. E.g. `help(rl.download_db)`__

_______

## 5 - Download receptor-ligand database
Provide a path to a table with receptor-ligand interactions. Will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

In [ ]:
rl.download_db(adata=adata,
               db_path=db_path,
               ligand_column='ligand_gene_symbol',
               receptor_column='receptor_gene_symbol',
               inplace=True,
               overwrite=False)

--------------

## 6 - Compute interactions
Create a table of receptor-ligand interactions between given groups. This table is used for plotting. Will be stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

In [ ]:
rl.calculate_interaction_table(adata=adata,
                               cluster_column=cluster_col,
                               gene_index=gene_col,
                               normalize=normalize,
                               inplace=True,
                               overwrite=overwrite)

--------------

## 7 - Plotting
<hr style="border:2px solid black"> </hr>

### 7.1 - Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

In [ ]:
rl.interaction_violin_plot(adata,
                           min_perc=violin_min_perc,
                           save=f"rl_{violin_out_suffix}.pdf",
                           figsize=violin_figsize,
                           dpi=dpi)

__________

### 7.2 - Network
Show a network graph of number of interactions between groups.

In [ ]:
rl.hairball(adata,
            min_perc=net_min_perc,
            interaction_score=net_interaction_score,
            interaction_perc=net_interaction_perc,
            save=f"rl_{net_out_suffix}.pdf",
            title=net_title,
            color_min=net_color_min,
            color_max=net_color_max,
            restrict_to=net_restrict_to,
            show_count=net_show_count
           )

__________

### 7.3 - Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: Receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

In [ ]:
rl.connectionPlot(adata=adata, 
                  restrict_to=con_restrict_to,
                  figsize=con_figsize,
                  dpi=dpi,
                  connection_alpha="interaction_score",
                  save=f"rl_{con_out_suffix}.pdf",
                  title=con_title,
                  receptor_cluster_col="receptor_cluster",
                  receptor_col="receptor_gene",
                  receptor_hue="receptor_score",
                  receptor_size="receptor_percent",
                  receptor_genes=con_rec_whitelist,
                  ligand_cluster_col="ligand_cluster",
                  ligand_col="ligand_gene",
                  ligand_hue="ligand_score",
                  ligand_size="ligand_percent",
                  ligand_genes=con_lig_whitelist,
                  filter=con_filter,
                  lw_multiplier=con_lw_multiplier,
                  wspace=con_wspace,
                  line_colors="rainbow")

__________

## 8 - Get interaction table

In [ ]:
interaction_table = rl.get_interactions(adata,
                                        min_perc = tab_min_perc,
                                        interaction_score = tab_interaction_score,
                                        interaction_perc = tab_interaction_perc,
                                        group_a = group_a,
                                        group_b = group_b,
                                        save = f"rl_{tab_out_suffix}.tsv")